In [53]:
import os
from input_reader import InputReader
from tracker import Tracker
import copy
import numpy as np
import traceback
from scipy.spatial import distance as dist
import pandas as pd

In [54]:
def swap_xy(corr):
    arr = copy.deepcopy(corr)

    arr[0], arr[1] = arr[1], arr[0]
    return arr

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    C = dist.euclidean(eye[0], eye[3])

    ear = (A + B) / (2.0 * C)
    return ear

In [55]:
max_threads = 4
model = 3
video_path = '.\\videos\\test_2.mp4'
raw_rgb = 0
width = 640
height = 360
fps = 0
use_dshowcapture_flag = True
silent = 0
SKIP_FIRST_FRAMES = 0

In [63]:
os.environ["OMP_NUM_THREADS"] = str(max_threads)

In [64]:
def process_video(video_path, raw_rgb, width, height, fps, use_dshowcapture_flag):
    failures = 0
    input_reader = InputReader(video_path, raw_rgb, width, height, fps, use_dshowcapture=use_dshowcapture_flag)
    COUNTER = 0
    TOTAL = 0
    current_frame = 1
    blink_start = 0
    blink_end = 0
    closeness = 0
    output_closeness = []
    output_blinks = []
    blink_info = (0,0)
    processed_frame = []
    frame_info_list = []
    lStart = 42
    lEnd = 48
    rStart = 36
    rEnd = 42
    ear_th = 0.21
    consec_th = 3
    up_to = None
    repeat = False
    while repeat or input_reader.is_open():
        ret, frame = input_reader.read()
        if not ret:
            if repeat:
                if need_reinit == 0:
                    need_reinit = 1
                continue
            else:
                break
        fps = input_reader.get_fps()
        frame_count = int(input_reader.get_frame())
        duration = frame_count/fps
        video_info_dict = {
            'fps': fps,
            'frame_count': frame_count,
            'duration(s)': duration
        }
        height, width, channels = frame.shape
        tracker = Tracker(width, height, threshold=None, max_threads=max_threads, max_faces=1, discard_after=10, scan_every=3, silent=False if 0 == 0 else True, model_type=model, model_dir=None, no_gaze=False if 1 != 0 and 3 != -1 else True, detection_threshold=0.02, use_retinaface=0, max_feature_updates=900, static_model=True if 1 == 1 else False, try_hard=0 == 1)
        try:
            faces, rmat = tracker.predict(frame)

            for f in faces:
                r_eye = [swap_xy(f.lms[36]), swap_xy(f.lms[37]), swap_xy(f.lms[38]), swap_xy(f.lms[39]), swap_xy(f.lms[40]), swap_xy(f.lms[41])]
                l_eye = [swap_xy(f.lms[42]), swap_xy(f.lms[43]), swap_xy(f.lms[44]), swap_xy(f.lms[45]), swap_xy(f.lms[46]), swap_xy(f.lms[47])]
                ear = np.average((eye_aspect_ratio(r_eye), eye_aspect_ratio(l_eye)))


                if ear < ear_th:
                    COUNTER += 1
                    closeness = 1
                    output_closeness.append(closeness)
                else:
                    if COUNTER >= consec_th:
                        TOTAL += 1
                        blink_start = current_frame - COUNTER
                        blink_end = current_frame - 1
                        blink_info = (blink_start, blink_end)
                        output_blinks.append(blink_info)
                    COUNTER = 0
                    closeness = 0
                    output_closeness.append(closeness)

                frame_info = {
                    'frame_no': current_frame,
                    'face_detected': 1,
                    'face_coordinates': 0,
                    'left_eye_coor': 0,
                    'right_eye_coor': 0,
                    'left_ear': eye_aspect_ratio(l_eye),
                    'right_ear': eye_aspect_ratio(r_eye),
                    'avg_ear': ear,
                    'closeness': closeness,
                    'blink_no': TOTAL,
                    'blink_start_frame': blink_start,
                    'blink_end_frame': blink_end,
                    'reserved_for_calibration': False
                }
                frame_info_list.append(frame_info)

                processed_frame.append(frame)
                current_frame += 1
                frame_info_df = pd.DataFrame(frame_info_list)

        except Exception as e:
            if e.__class__ == KeyboardInterrupt:
                if silent == 0:
                    print("Quitting")
                break
            traceback.print_exc()
            failures += 1
            if failures > 30:
                break
    file_name = os.path.basename(video_path)
    output_str = 'Processing {} has done.\n\n'.format(file_name)
    return frame_info_df, output_closeness, output_blinks, processed_frame, video_info_dict, output_str

In [65]:
frame_info_df, closeness_predictions, blink_predictions, frames, video_info, scores_string = process_video(video_path, raw_rgb, width, height, fps, use_dshowcapture_flag)

Took 31.33ms (detect: 10.69ms, crop: 1.51ms, track: 16.41ms, 3D points: 2.66ms)
Took 35.98ms (detect: 11.54ms, crop: 1.52ms, track: 19.58ms, 3D points: 3.29ms)
Took 46.60ms (detect: 14.26ms, crop: 1.83ms, track: 27.70ms, 3D points: 2.73ms)
Took 37.23ms (detect: 11.33ms, crop: 2.52ms, track: 20.67ms, 3D points: 2.62ms)
Took 45.15ms (detect: 18.73ms, crop: 1.53ms, track: 21.73ms, 3D points: 3.10ms)
Took 39.01ms (detect: 13.59ms, crop: 1.44ms, track: 21.06ms, 3D points: 2.87ms)
Took 42.39ms (detect: 16.30ms, crop: 2.12ms, track: 21.13ms, 3D points: 2.78ms)
Took 42.27ms (detect: 12.65ms, crop: 4.57ms, track: 21.60ms, 3D points: 3.37ms)
Took 34.56ms (detect: 10.54ms, crop: 1.58ms, track: 19.70ms, 3D points: 2.68ms)
Took 38.10ms (detect: 11.71ms, crop: 1.56ms, track: 21.95ms, 3D points: 2.82ms)
Took 36.54ms (detect: 13.67ms, crop: 1.60ms, track: 17.66ms, 3D points: 3.56ms)
Took 30.70ms (detect: 10.15ms, crop: 1.48ms, track: 16.46ms, 3D points: 2.56ms)
Took 30.68ms (detect: 10.35ms, crop: 1.4

Took 33.19ms (detect: 11.58ms, crop: 1.77ms, track: 16.87ms, 3D points: 2.91ms)
Took 37.07ms (detect: 11.43ms, crop: 1.95ms, track: 20.69ms, 3D points: 2.95ms)
Took 40.45ms (detect: 13.64ms, crop: 1.71ms, track: 21.76ms, 3D points: 3.26ms)
Took 56.15ms (detect: 20.72ms, crop: 2.08ms, track: 29.46ms, 3D points: 3.44ms)
Took 41.62ms (detect: 12.69ms, crop: 1.75ms, track: 24.07ms, 3D points: 3.05ms)
Took 36.47ms (detect: 11.98ms, crop: 1.85ms, track: 19.50ms, 3D points: 3.09ms)
Took 35.68ms (detect: 12.57ms, crop: 1.61ms, track: 18.01ms, 3D points: 3.42ms)
Took 34.63ms (detect: 11.96ms, crop: 1.57ms, track: 18.05ms, 3D points: 3.00ms)
Took 37.90ms (detect: 12.63ms, crop: 1.55ms, track: 20.69ms, 3D points: 2.97ms)
Took 84.58ms (detect: 28.09ms, crop: 2.53ms, track: 44.81ms, 3D points: 9.03ms)
Took 77.45ms (detect: 18.62ms, crop: 1.83ms, track: 43.14ms, 3D points: 13.78ms)
Took 47.86ms (detect: 16.35ms, crop: 1.90ms, track: 25.74ms, 3D points: 3.80ms)
Took 38.33ms (detect: 12.16ms, crop: 1.

Took 77.95ms (detect: 29.21ms, crop: 4.22ms, track: 38.20ms, 3D points: 6.20ms)
Took 64.85ms (detect: 21.13ms, crop: 1.90ms, track: 35.70ms, 3D points: 6.03ms)
Took 51.20ms (detect: 16.96ms, crop: 1.87ms, track: 26.39ms, 3D points: 5.90ms)
Took 60.00ms (detect: 21.48ms, crop: 2.19ms, track: 32.46ms, 3D points: 3.78ms)
Took 115.60ms (detect: 31.93ms, crop: 4.14ms, track: 74.45ms, 3D points: 4.95ms)
Took 52.72ms (detect: 16.91ms, crop: 1.79ms, track: 30.04ms, 3D points: 3.91ms)
Took 49.45ms (detect: 15.22ms, crop: 1.71ms, track: 29.01ms, 3D points: 3.42ms)
Took 45.91ms (detect: 14.22ms, crop: 1.59ms, track: 26.68ms, 3D points: 3.34ms)
Took 48.89ms (detect: 16.25ms, crop: 1.63ms, track: 25.79ms, 3D points: 5.15ms)
Took 40.82ms (detect: 13.72ms, crop: 1.68ms, track: 22.26ms, 3D points: 3.10ms)
Took 70.83ms (detect: 24.77ms, crop: 2.56ms, track: 38.65ms, 3D points: 4.76ms)
Took 81.79ms (detect: 34.45ms, crop: 3.02ms, track: 39.35ms, 3D points: 4.83ms)
Took 87.34ms (detect: 37.21ms, crop: 3.

Took 60.63ms (detect: 17.68ms, crop: 0.88ms, track: 35.46ms, 3D points: 6.53ms)
Took 35.29ms (detect: 12.58ms, crop: 0.77ms, track: 18.66ms, 3D points: 3.21ms)
Took 33.04ms (detect: 11.28ms, crop: 1.00ms, track: 17.85ms, 3D points: 2.84ms)
Took 30.78ms (detect: 11.44ms, crop: 0.75ms, track: 15.76ms, 3D points: 2.77ms)
Took 30.87ms (detect: 11.41ms, crop: 0.79ms, track: 15.84ms, 3D points: 2.78ms)
Took 35.90ms (detect: 11.54ms, crop: 0.78ms, track: 20.68ms, 3D points: 2.85ms)
Took 44.03ms (detect: 17.27ms, crop: 1.25ms, track: 22.62ms, 3D points: 2.83ms)
Took 41.94ms (detect: 15.22ms, crop: 2.25ms, track: 21.21ms, 3D points: 3.18ms)
Took 36.60ms (detect: 12.48ms, crop: 0.75ms, track: 20.48ms, 3D points: 2.83ms)
Took 34.07ms (detect: 12.00ms, crop: 0.74ms, track: 18.52ms, 3D points: 2.76ms)
Took 31.94ms (detect: 11.50ms, crop: 0.74ms, track: 16.76ms, 3D points: 2.87ms)
Took 30.80ms (detect: 11.23ms, crop: 0.82ms, track: 15.90ms, 3D points: 2.79ms)
Took 29.15ms (detect: 10.89ms, crop: 0.7

Took 31.58ms (detect: 11.81ms, crop: 0.80ms, track: 16.06ms, 3D points: 2.85ms)
Took 31.62ms (detect: 11.28ms, crop: 0.85ms, track: 16.56ms, 3D points: 2.88ms)
Took 31.46ms (detect: 11.98ms, crop: 0.79ms, track: 15.69ms, 3D points: 2.95ms)
Took 87.40ms (detect: 31.12ms, crop: 1.23ms, track: 47.31ms, 3D points: 7.65ms)
Took 75.07ms (detect: 22.69ms, crop: 1.67ms, track: 44.75ms, 3D points: 5.82ms)
Took 51.06ms (detect: 16.56ms, crop: 0.99ms, track: 28.79ms, 3D points: 4.62ms)
Took 49.41ms (detect: 18.00ms, crop: 1.17ms, track: 26.65ms, 3D points: 3.48ms)
Took 44.36ms (detect: 14.21ms, crop: 1.01ms, track: 24.74ms, 3D points: 4.30ms)
Took 34.94ms (detect: 12.89ms, crop: 1.14ms, track: 17.81ms, 3D points: 3.02ms)
Took 36.55ms (detect: 11.84ms, crop: 0.96ms, track: 20.75ms, 3D points: 2.94ms)
Took 34.30ms (detect: 12.46ms, crop: 0.97ms, track: 18.22ms, 3D points: 2.59ms)
Took 44.53ms (detect: 18.71ms, crop: 0.96ms, track: 21.93ms, 3D points: 2.86ms)
Took 32.51ms (detect: 11.83ms, crop: 0.8

...

Took 31.26ms (detect: 11.27ms, crop: 1.40ms, track: 16.09ms, 3D points: 2.45ms)
Took 32.97ms (detect: 11.53ms, crop: 1.30ms, track: 17.56ms, 3D points: 2.52ms)
Took 30.64ms (detect: 11.02ms, crop: 1.42ms, track: 15.75ms, 3D points: 2.40ms)
Took 31.52ms (detect: 11.36ms, crop: 1.40ms, track: 16.14ms, 3D points: 2.56ms)
Took 31.67ms (detect: 11.41ms, crop: 1.44ms, track: 16.33ms, 3D points: 2.44ms)
Took 31.39ms (detect: 11.45ms, crop: 1.39ms, track: 16.30ms, 3D points: 2.20ms)
Took 31.45ms (detect: 10.88ms, crop: 1.47ms, track: 16.64ms, 3D points: 2.40ms)
Took 35.23ms (detect: 11.19ms, crop: 1.58ms, track: 19.95ms, 3D points: 2.44ms)
Took 31.42ms (detect: 11.47ms, crop: 1.40ms, track: 15.98ms, 3D points: 2.50ms)
Took 46.94ms (detect: 11.84ms, crop: 1.44ms, track: 31.30ms, 3D points: 2.31ms)
Took 35.90ms (detect: 11.54ms, crop: 1.51ms, track: 20.14ms, 3D points: 2.65ms)
Took 34.89ms (detect: 11.41ms, crop: 1.59ms, track: 19.44ms, 3D points: 2.38ms)
Took 30.82ms (detect: 11.11ms, crop: 1.3

Took 52.62ms (detect: 17.55ms, crop: 1.69ms, track: 30.08ms, 3D points: 3.17ms)
Took 38.42ms (detect: 14.37ms, crop: 1.84ms, track: 19.09ms, 3D points: 3.06ms)
Took 38.09ms (detect: 12.34ms, crop: 1.59ms, track: 21.14ms, 3D points: 2.95ms)
Took 32.62ms (detect: 11.47ms, crop: 1.58ms, track: 16.65ms, 3D points: 2.86ms)
Took 55.95ms (detect: 17.28ms, crop: 1.81ms, track: 32.32ms, 3D points: 4.47ms)
Took 43.11ms (detect: 13.65ms, crop: 2.26ms, track: 23.76ms, 3D points: 3.37ms)
Took 40.09ms (detect: 13.22ms, crop: 1.83ms, track: 21.83ms, 3D points: 3.15ms)
Took 36.40ms (detect: 12.89ms, crop: 1.63ms, track: 18.76ms, 3D points: 3.06ms)
Took 34.07ms (detect: 11.66ms, crop: 1.64ms, track: 17.74ms, 3D points: 2.97ms)
Took 34.19ms (detect: 11.30ms, crop: 1.64ms, track: 18.06ms, 3D points: 3.14ms)
Took 33.39ms (detect: 11.56ms, crop: 1.62ms, track: 17.43ms, 3D points: 2.72ms)
Took 39.98ms (detect: 12.84ms, crop: 1.77ms, track: 22.34ms, 3D points: 2.97ms)
Took 34.48ms (detect: 12.32ms, crop: 1.6

Took 33.49ms (detect: 12.29ms, crop: 1.36ms, track: 17.24ms, 3D points: 2.54ms)
Took 50.46ms (detect: 17.07ms, crop: 1.33ms, track: 29.38ms, 3D points: 2.62ms)
Took 59.07ms (detect: 24.91ms, crop: 1.65ms, track: 27.93ms, 3D points: 4.50ms)
Took 114.83ms (detect: 40.95ms, crop: 5.25ms, track: 64.57ms, 3D points: 3.92ms)
Took 57.01ms (detect: 21.84ms, crop: 1.41ms, track: 30.80ms, 3D points: 2.88ms)
Took 30.73ms (detect: 11.39ms, crop: 1.28ms, track: 15.67ms, 3D points: 2.34ms)
Took 33.68ms (detect: 11.97ms, crop: 1.33ms, track: 18.15ms, 3D points: 2.15ms)
Took 31.92ms (detect: 11.76ms, crop: 1.46ms, track: 16.22ms, 3D points: 2.42ms)
Took 33.74ms (detect: 12.18ms, crop: 1.31ms, track: 17.48ms, 3D points: 2.69ms)
Took 105.03ms (detect: 36.54ms, crop: 2.36ms, track: 62.92ms, 3D points: 3.14ms)
Took 122.84ms (detect: 27.82ms, crop: 5.45ms, track: 81.94ms, 3D points: 7.55ms)
Took 33.42ms (detect: 12.18ms, crop: 1.46ms, track: 16.98ms, 3D points: 2.73ms)
Took 32.52ms (detect: 11.95ms, crop: 

Took 35.96ms (detect: 12.66ms, crop: 1.23ms, track: 19.51ms, 3D points: 2.50ms)
Took 62.82ms (detect: 15.65ms, crop: 1.20ms, track: 38.21ms, 3D points: 7.50ms)
Took 49.35ms (detect: 16.49ms, crop: 1.09ms, track: 28.82ms, 3D points: 2.88ms)
Took 58.45ms (detect: 21.56ms, crop: 3.31ms, track: 29.83ms, 3D points: 3.69ms)
Took 52.47ms (detect: 17.15ms, crop: 1.41ms, track: 30.53ms, 3D points: 3.30ms)
Took 52.42ms (detect: 18.67ms, crop: 1.50ms, track: 27.69ms, 3D points: 4.48ms)
Took 44.27ms (detect: 12.88ms, crop: 1.44ms, track: 27.33ms, 3D points: 2.56ms)
Took 53.11ms (detect: 17.84ms, crop: 1.45ms, track: 30.55ms, 3D points: 3.21ms)
Took 61.47ms (detect: 26.54ms, crop: 1.33ms, track: 30.11ms, 3D points: 3.41ms)
Took 38.20ms (detect: 13.67ms, crop: 1.45ms, track: 20.51ms, 3D points: 2.52ms)
Took 42.76ms (detect: 16.83ms, crop: 2.18ms, track: 20.73ms, 3D points: 2.96ms)
Took 55.00ms (detect: 18.93ms, crop: 1.80ms, track: 30.38ms, 3D points: 3.82ms)
Took 50.57ms (detect: 14.70ms, crop: 1.7

KeyboardInterrupt: 